In [ ]:
#import important modules
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn as skl
import json
import pickle
import re
import pprint
import datetime
import statistics as st

In [ ]:
import sqlite3
high_performance_mode = False #Changed this to default false
db_file_path = "db.db" #CHANGE THIS
from urllib.parse import unquote
from urllib.parse import quote

In [ ]:
##DO NOT TOUCH THESE
db = sqlite3.connect(":memory:")
db2 = sqlite3.connect(db_file_path)
##DO NOT TOUCH THESE
if high_performance_mode:
    db2.backup(db)
    db2.close()
else:
    db = db2
db.execute('PRAGMA journal_mode = MEMORY')
#Rollback journal in mem so that indexing is faster and crashes are catastrophic >:D
db.execute('PRAGMA cache_size = 536870912') #<- 512MB in B
#This is dinamically resized but the default is really low so let it grow!
db.execute('PRAGMA temp_store = MEMORY')
#Anything that needs to be generated, do it in mem because faster
db.execute('PRAGMA locking_mode = EXCLUSIVE')
#Remove lock/unlock overhead let it zoom
print("Loaded")

In [ ]:
c = db.cursor()
class Helper:
      def __init__(self, db):
        self.db = db;
        #If anybody wants to use this Google "SQLite IN"
        self.airline_names = {20626359: 'virginatlantic', 18332190: 'British Airways', 22536055: 'American Airlines', 1542862735: 'Ryanair', 38676903: 'easyJet', 124476322: 'Lufthansa', 218730857: 'Qantas', 106062176: 'Air France', 56377143: 'Royal Dutch Airlines', 45621423: 'SocialLogin%40etihad.ae', 253340062: 'Singapore Airlines', 26223583: 'airberlin', 2182373406: 'airberlin Assist'}
        self.airline_ids = [56377143, 106062176, 18332190, 22536055, 124476322, 26223583, 2182373406, 38676903, 1542862735, 253340062, 218730857, 45621423, 20626359]
      def exec_custom(self, query: str):
        return self.db.execute(query)
      def get_vars(self, table: str):
        return self.db.execute("pragma table_info('{0}');".format(table))
      def get_tables(self): 
        return self.db.execute("SELECT name FROM sqlite_master WHERE type ='table' AND name NOT LIKE 'sqlite_%';")
      def get_tweet(self, id: int, vars, filter):
        return self.db.execute(self.build_query('tweets', vars, filter))
      def get_user_tweets(self, id: int):
        return self.db.execute(self.build_query('tweets', '*', [['user', id]]))
      def get_airline_tweets_by_mention(self, id: int, vars):
        return self.db.execute(self.build_query('mentions', vars, "WHERE user_id = '{0}'".format(id)))
      def get_airline_tweets_by_user_id_in_tweet(self, id: int, vars):
        return self.db.execute(self.build_query('tweets', vars, "WHERE user = '{0}'".format(id)))
      def build_query(self, table, return_values, search_values, raw_filter = 0):
        def createLimit(limit: list):
            return limit[0] + " = " + "'{0}'".format(limit[1])
        #You'd have to sort and/or/* search values and then apply them accordingly because of how the .join function works and im not about that life
        #Or use the list comprehension output to process the logic behind it (but usually complex things require brackets and f that)
        if(type(search_values) == list):
            query = "select " + ",".join(return_values) + " from " + table + " where " + ' and '.join([createLimit(limit) for limit in search_values])
        else:
            query = "select " + ",".join(return_values) + " from " + table + " where " + search_values
        return query


In [ ]:
hp = Helper(c)

In [ ]:
#See all tables
hp_t = Helper(db)
for table_name in hp_t.get_tables():
    print('<[{0}]>'.format(table_name[0]))
    for column_name in hp_t.get_vars(table_name[0]):
        print("    {0}".format(column_name[1]))

In [ ]:
airlines = {'KLM':56377143, 
            'AirFrance': 106062176, 
            'British_Airways': 18332190, 
            'AmericanAir': 22536055, 
           'Lufthansa': 124476322, 
           'AirBerlin': 26223583, 
           'AirBerlinAssist': 2182373406, 
           'easyJet': 38676903, 
           'RyanAir': 1542862735, 
           'SingaporeAir': 253340062, 
           'Qantas': 218730857, 
           'EtihadAirways': 45621423, 
           'VirginAtlantic': 20626359}

In [ ]:
air_count = pickle.load(open("save2.p", "rb"))

In [ ]:
#noticed there are two keys we don't need
del air_count['AirBerlin']['nr_convs']
del air_count['AirBerlinAssist']['nr_convs']

- KLM received about 133 000 tweets from users (whether directly or as a reply)
- did they reply more to famous users? (famous defined as users with more than 5000 followers)

In [ ]:
q1 = """
SELECT COUNT(DISTINCT(tweets.id))
FROM users, tweets
WHERE users.id = tweets.user AND users.id = 56377143 AND tweets.in_reply_to_user_id != 'null'
AND tweets.in_reply_to_user_id IN (SELECT id FROM users WHERE followers_count > 5000);
"""
for row in hp.exec_custom(q1):
    print(row)

# A-B-A structure for the sentiment analysis

First, find the tweets from users that get a reply later on (only KLM)

In [ ]:
unfiltered_a_q = """
SELECT DISTINCT(id) as 'start', user as 'user_id'
FROM tweets
WHERE tweets.user != 56377143 AND tweets.id IN ({0}) AND tweets.in_reply_to_user_id = 'null' 
AND id IN (SELECT DISTINCT(in_reply_to_status_id) FROM tweets WHERE user = 56377143 AND in_reply_to_status_id != 'null')
""".format(','.join([str(x) for x in air_count['KLM']['ids']]))

unfiltered_a = pd.read_sql_query(unfiltered_a_q, db)
unfiltered_a #these are the tweets directly sent from users that get a reply

We have to filter these tweets to those that end in a ABA structured converstion 

   - going backwards from the third (A) tweet to klm's reply (B) 

In [ ]:
b_tweets_id = """(SELECT DISTINCT(id) FROM tweets WHERE tweets.user = 56377143 AND in_reply_to_status_id IN ({0}))""".format( ','.join([str(x) for x in unfiltered_a['start']]))
b_tweets_in_reply_user = """(SELECT DISTINCT(in_reply_to_user_id) FROM tweets WHERE tweets.user = 56377143 AND in_reply_to_status_id IN ({0}))""".format( ','.join([str(x) for x in unfiltered_a['start']]))
a_tweets = """(SELECT DISTINCT(in_reply_to_status_id) FROM tweets WHERE tweets.user = 56377143 AND in_reply_to_status_id IN ({0}))""".format( ','.join([str(x) for x in unfiltered_a['start']]))
backward_a_b_q = """
SELECT DISTINCT(id) as 'change', user as 'user_id', in_reply_to_status_id as 'klms reply'
FROM tweets
WHERE tweets.user != 56377143 AND tweets.id IN ({0}) AND tweets.user IN {1}
AND tweets.in_reply_to_status_id IN {2}
""".format(','.join([str(x) for x in air_count['KLM']['ids']]), b_tweets_in_reply_user, b_tweets_id)
backward_a_b = pd.read_sql_query(backward_a_b_q, db)
backward_a_b #klms reply to original user tweet and a users reply to that reply

- first (A) tweets from users that ended in a conversation

In [ ]:
a_tweets_q = """
SELECT in_reply_to_status_id as 'start'
FROM tweets
WHERE id IN ({0})
""".format(','.join([str(x) for x in backward_a_b['klms reply']]))
a_tweets = pd.read_sql_query(a_tweets_q, db)
a_tweets #filtered original first tweets that get a reply and reply to a reply 